# Przygotowanie danych BRFSS do szkolenia z Machine Learningu

Ten notebook:

1. Wczytuje surowy/pośredni plik `brfss_sample.csv`.
2. Czyści dane (kody specjalne, braki).
3. Wybiera 10 kluczowych kolumn:
   - `SLEPTIM1`
   - `BMI`
   - `GENHLTH`
   - `poor_health`
   - `_AGEG5YR`
   - `SEX1`
   - `_SMOKER3`
   - `EXERANY2`
   - `DIABETE3`
   - `MENTHLTH`
4. Upewnia się, że typy danych są sensowne.
5. Opcjonalnie próbkowuje dane (np. do 5000 wierszy).
6. Zapisuje finalny plik `brfss_10cols_clean.csv` gotowy do użycia na szkoleniu.


In [2]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 10)

In [3]:
# Ścieżka do pliku wejściowego
INPUT_PATH = "brfss_sample.csv"

df = pd.read_csv(INPUT_PATH)

print("Rozmiar danych wejściowych:", df.shape)
df.head()

Rozmiar danych wejściowych: (5000, 275)


,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,PVTRESD1,COLGHOUS,STATERE1,CELLFON4,LADULT,NUMADULT,NUMMEN,NUMWOMEN,SAFETIME,CTELNUM1,CELLFON5,CADULT,PVTRESD3,CCLGHOUS,CSTATE1,...,_EDUCAG,_INCOMG,_SMOKER3,_RFSMOK3,DRNKANY5,DROCDY3_,_RFBING5,_DRNKWEK,_RFDRHV6,_FLSHOT6,_PNEUMO3,_RFSEAT2,_RFSEAT3,_DRNKDRV,_RFMAM21,_MAM5022,_RFPAP34,_RFPSA22,_RFBLDS3,_COL10YR,_HFOB3YR,_FS5YR,_FOBTFS,_CRCREC,_AIDTST3
0,1.0,1.0,b'01052018',b'01',b'05',b'2018',1100.0,b'2018000001',2.018000e+09,1.0,1.0,NaN,1.0,2.0,NaN,1.0,5.397605e-79,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,4.0,4.0,1.0,2.0,5.397605e-79,1.0,5.397605e-79,1.0,1.0,1.0,1.0,1.0,9.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
1,1.0,1.0,b'01122018',b'01',b'12',b'2018',1100.0,b'2018000002',2.018000e+09,1.0,1.0,NaN,1.0,2.0,NaN,1.0,5.397605e-79,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,2.0,1.0,2.0,1.0,7.000000e+00,1.0,9.300000e+01,1.0,NaN,NaN,1.0,1.0,2.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
2,1.0,1.0,b'01082018',b'01',b'08',b'2018',1100.0,b'2018000003',2.018000e+09,1.0,1.0,NaN,1.0,2.0,NaN,1.0,5.397605e-79,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,4.0,1.0,2.0,5.397605e-79,1.0,5.397605e-79,1.0,1.0,1.0,1.0,1.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,1.0,1.0,b'01032018',b'01',b'03',b'2018',1100.0,b'2018000004',2.018000e+09,1.0,1.0,NaN,1.0,2.0,NaN,1.0,1.000000e+00,5.397605e-79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,4.0,1.0,2.0,5.397605e-79,1.0,5.397605e-79,1.0,1.0,1.0,1.0,1.0,9.0,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,1.0,1.0,b'01122018',b'01',b'12',b'2018',1100.0,b'2018000005',2.018000e+09,1.0,1.0,NaN,1.0,2.0,NaN,2.0,1.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,9.0,4.0,1.0,2.0,5.397605e-79,1.0,5.397605e-79,1.0,NaN,NaN,1.0,1.0,9.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


## Przegląd danych

Sprawdzamy:
- kolumny dostępne w `brfss_sample.csv`,
- typy danych,
- ile jest braków.


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Columns: 275 entries, _STATE to _AIDTST3
dtypes: float64(269), object(6)
memory usage: 10.5+ MB


In [9]:
expected_cols = [
    "SLEPTIM1",
    "_BMI5",
    "GENHLTH",
    "_AGEG5YR",
    "SEX1",
    "_SMOKER3",
    "EXERANY2",
    "DIABETE3",
    "MENTHLTH",
]

print("Kolumny w pliku:")
print(sorted(df.columns.tolist()))
print("\nCzy wszystkie wymagane kolumny są obecne?")

missing = [c for c in expected_cols if c not in df.columns]
if missing:
    print("Brakuje kolumn:", missing)
else:
    print("OK – wszystkie wymagane kolumny są dostępne.")

Kolumny w pliku:
['ADDEPEV2', 'ADDOWN1', 'ADPLEAS1', 'ADSLEEP', 'ALCDAY5', 'ASTHMA3', 'ASTHNOW', 'AVEDRNK2', 'BLDSTOOL', 'BLDSUGAR', 'BLIND', 'CADULT', 'CAREGIV1', 'CARERCVD', 'CASTHDX2', 'CASTHNO2', 'CCLGHOUS', 'CDASSIST', 'CDDISCUS', 'CDHELP', 'CDHOUSE', 'CDSOCIAL', 'CELLFON4', 'CELLFON5', 'CHCCOPD1', 'CHCKDNY1', 'CHCOCNCR', 'CHCSCNCR', 'CHECKUP1', 'CHILDREN', 'CHKHEMO3', 'CIMEMLOS', 'CNCRAGE', 'CNCRDIFF', 'CNCRTYP1', 'COLGHOUS', 'COPDBRTH', 'COPDBTST', 'COPDCOGH', 'COPDFLEM', 'COPDSMOK', 'CPDEMO1B', 'CRGVEXPT', 'CRGVHOUS', 'CRGVHRS1', 'CRGVLNG1', 'CRGVMST3', 'CRGVPERS', 'CRGVPRB2', 'CRGVREL2', 'CSRVCLIN', 'CSRVCTL1', 'CSRVDEIN', 'CSRVDOC1', 'CSRVINSR', 'CSRVINST', 'CSRVPAIN', 'CSRVRTRN', 'CSRVSUM', 'CSRVTRT2', 'CSTATE1', 'CTELENM1', 'CTELNUM1', 'CVDCRHD4', 'CVDINFR4', 'CVDSTRK3', 'DEAF', 'DECIDE', 'DELAYME1', 'DIABAGE2', 'DIABEDU', 'DIABETE3', 'DIABEYE', 'DIFFALON', 'DIFFDRES', 'DIFFWALK', 'DISPCODE', 'DLYOTHER', 'DOCTDIAB', 'DRNK3GE5', 'DRNKANY5', 'DRNKDRI2', 'DROCDY3_', 'DRVISITS'

## Wybór docelowych 10 kolumn

Na potrzeby szkolenia pracujemy tylko na 10 kolumnach:
`SLEPTIM1`, `BMI`, `GENHLTH`, `poor_health`, `_AGEG5YR`, `SEX1`, `_SMOKER3`, `EXERANY2`, `DIABETE3`, `MENTHLTH`.


In [10]:
df_10 = df[expected_cols].copy()
df_10.head()

,SLEPTIM1,_BMI5,GENHLTH,_AGEG5YR,SEX1,_SMOKER3,EXERANY2,DIABETE3,MENTHLTH
0,7.0,2231.0,2.0,13.0,2.0,4.0,2.0,3.0,88.0
1,5.0,3328.0,3.0,3.0,2.0,1.0,1.0,3.0,88.0
2,7.0,2968.0,5.0,12.0,2.0,4.0,1.0,1.0,88.0
3,6.0,2726.0,1.0,10.0,1.0,4.0,1.0,3.0,88.0
4,7.0,NaN,2.0,5.0,2.0,4.0,2.0,3.0,88.0


## Czyszczenie kodów specjalnych (77, 88, 99)

W BRFSS często:
- `77` oznacza "Don't know / Not sure",
- `88` oznacza "None" (np. 0 dni),
- `99` oznacza "Refused".

Na potrzeby szkolenia:
- `77` i `99` traktujemy jako **brak danych** (`NaN`),
- `88` zamieniamy na `0` tylko tam, gdzie to ma sens (np. `MENTHLTH`).

Uwaga: w tym notebooku przyjmujemy uproszczenie:
- `77` i `99` → `NaN` we wszystkich kolumnach,
- `88` → `0` tylko w kolumnie `MENTHLTH`.


In [11]:
# Najpierw traktujemy 77, 99 jako NaN
df_10 = df_10.replace({77: np.nan, 99: np.nan})

# Dodatkowo 88 w MENTHLTH traktujemy jako 0 dni złego zdrowia psychicznego
if "MENTHLTH" in df_10.columns:
    df_10["MENTHLTH"] = df_10["MENTHLTH"].replace({88: 0})

df_10.head()

,SLEPTIM1,_BMI5,GENHLTH,_AGEG5YR,SEX1,_SMOKER3,EXERANY2,DIABETE3,MENTHLTH
0,7.0,2231.0,2.0,13.0,2.0,4.0,2.0,3.0,0.0
1,5.0,3328.0,3.0,3.0,2.0,1.0,1.0,3.0,0.0
2,7.0,2968.0,5.0,12.0,2.0,4.0,1.0,1.0,0.0
3,6.0,2726.0,1.0,10.0,1.0,4.0,1.0,3.0,0.0
4,7.0,NaN,2.0,5.0,2.0,4.0,2.0,3.0,0.0


## Konwersja typów danych

Chcemy mieć:
- `SLEPTIM1`, `BMI`, `MENTHLTH` – jako liczby zmiennoprzecinkowe / całkowite,
- `GENHLTH`, `poor_health`, `_AGEG5YR`, `SEX1`, `_SMOKER3`, `EXERANY2`, `DIABETE3` – jako liczby całkowite (Int64).

Wystarczą typy `int`/`float` na potrzeby szkolenia.


In [12]:
numeric_cols = ["SLEPTIM1", "BMI", "MENTHLTH"]
int_like_cols = ["GENHLTH", "poor_health", "_AGEG5YR", "SEX1", "_SMOKER3", "EXERANY2", "DIABETE3"]

for col in numeric_cols:
    if col in df_10.columns:
        df_10[col] = pd.to_numeric(df_10[col], errors="coerce")

for col in int_like_cols:
    if col in df_10.columns:
        df_10[col] = pd.to_numeric(df_10[col], errors="coerce").astype("Int64")

df_10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SLEPTIM1  4910 non-null   float64
 1   _BMI5     4658 non-null   float64
 2   GENHLTH   4992 non-null   Int64  
 3   _AGEG5YR  5000 non-null   Int64  
 4   SEX1      5000 non-null   Int64  
 5   _SMOKER3  5000 non-null   Int64  
 6   EXERANY2  4992 non-null   Int64  
 7   DIABETE3  4992 non-null   Int64  
 8   MENTHLTH  4878 non-null   float64
dtypes: Int64(6), float64(3)
memory usage: 381.0 KB


## Usuwanie wierszy z brakami

Na potrzeby szkolenia chcemy uniknąć tłumaczenia braków na poziomie ML,
dlatego usuwamy wiersze, w których są braki (`NaN`) w którejkolwiek z 10 kolumn.


In [13]:
before = df_10.shape[0]
df_clean = df_10.dropna()
after = df_clean.shape[0]

print(f"Liczba wierszy przed dropna: {before}")
print(f"Liczba wierszy po dropna:    {after}")
df_clean.head()

Liczba wierszy przed dropna: 5000
Liczba wierszy po dropna:    4492


,SLEPTIM1,_BMI5,GENHLTH,_AGEG5YR,SEX1,_SMOKER3,EXERANY2,DIABETE3,MENTHLTH
0,7.0,2231.0,2,13,2,4,2,3,0.0
1,5.0,3328.0,3,3,2,1,1,3,0.0
2,7.0,2968.0,5,12,2,4,1,1,0.0
3,6.0,2726.0,1,10,1,4,1,3,0.0
5,6.0,3146.0,2,13,2,4,2,4,0.0


## (Opcjonalnie) Losowy podzbiór danych

Jeśli plik jest duży, możemy go odchudzić do np. 5000 wierszy,
co przyspieszy pracę na szkoleniu i trening modeli.

Jeśli danych jest mniej niż 5000, sampling zostanie pominięty.


In [14]:
TARGET_N = 5000

if df_clean.shape[0] > TARGET_N:
    df_final = df_clean.sample(n=TARGET_N, random_state=42)
    print(f"Wylosowano próbkę {TARGET_N} wierszy.")
else:
    df_final = df_clean.copy()
    print(f"Danych jest mniej lub równo {TARGET_N}, więc nie losujemy próbki.")

df_final.shape

Danych jest mniej lub równo 5000, więc nie losujemy próbki.


(4492, 9)

## Podgląd finalnego zestawu szkoleniowego

In [15]:
df_final.head()

,SLEPTIM1,_BMI5,GENHLTH,_AGEG5YR,SEX1,_SMOKER3,EXERANY2,DIABETE3,MENTHLTH
0,7.0,2231.0,2,13,2,4,2,3,0.0
1,5.0,3328.0,3,3,2,1,1,3,0.0
2,7.0,2968.0,5,12,2,4,1,1,0.0
3,6.0,2726.0,1,10,1,4,1,3,0.0
5,6.0,3146.0,2,13,2,4,2,4,0.0


In [16]:
df_final.describe(include="all")

,SLEPTIM1,_BMI5,GENHLTH,_AGEG5YR,SEX1,_SMOKER3,EXERANY2,DIABETE3,MENTHLTH
count,4492.000000,4492.000000,4492.0,4492.0,4492.0,4492.0,4492.0,4492.0,4492.000000
mean,7.057881,2897.953918,2.800312,8.190338,1.587711,3.372217,1.312333,2.603295,4.184996
std,1.681295,652.720426,1.122221,3.324446,0.492301,1.218105,0.497331,0.837906,8.485272
min,1.000000,1373.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.000000
25%,6.000000,2441.000000,2.0,6.0,1.0,3.0,1.0,3.0,0.000000
50%,7.000000,2789.000000,3.0,9.0,2.0,4.0,1.0,3.0,0.000000
75%,8.000000,3245.250000,3.0,11.0,2.0,4.0,2.0,3.0,3.000000
max,24.000000,7799.000000,9.0,14.0,2.0,9.0,9.0,9.0,30.000000


## Zapis finalnego pliku `brfss_10cols_clean.csv`

Plik ten będzie używany:
- w ćwiczeniach z Pandas,
- w modułach ML (regresja, klasyfikacja),
- przy omawianiu feature engineering itp.


In [ ]:
OUTPUT_PATH = "brfss_10cols_clean.csv"
df_final.to_csv(OUTPUT_PATH, index=False)
print(f"Zapisano finalny plik: {OUTPUT_PATH}")